In [228]:
import pandas as pd
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

%matplotlib inline

In [229]:
works_custs = pd.read_csv('AdvWorksCusts.csv')
works_custs.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
print(works_custs.columns)

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome'],
      dtype='object')


In [230]:
works_custs.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,4/8/66,Bachelors,Professional,M,M,1,0,0,2,137947
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,5/14/65,Bachelors,Professional,M,S,0,1,3,3,101141
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,8/12/65,Bachelors,Professional,M,M,1,1,3,3,91945
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,2/15/68,Bachelors,Professional,F,S,0,1,0,0,86688
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,8/8/68,Bachelors,Professional,F,S,1,4,5,5,92771


In [231]:
bike_buyer = pd.read_csv('AW_BikeBuyer.csv')
bike_buyer.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
print(bike_buyer.columns)

Index(['CustomerID', 'BikeBuyer'], dtype='object')


In [232]:
bike_buyer.head()

,CustomerID,BikeBuyer
0,11000,0
1,11001,1
2,11002,0
3,11003,0
4,11004,1


In [233]:
works_custs_bike_buyer = pd.merge(works_custs, bike_buyer, how='left', on='CustomerID')
print(works_custs_bike_buyer.columns)
print(works_custs_bike_buyer)

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'BikeBuyer'],
      dtype='object')
       CustomerID Title   FirstName MiddleName    LastName Suffix  \
0           11000   NaN         Jon          V        Yang    NaN   
1           11001   NaN      Eugene          L       Huang    NaN   
2           11002   NaN       Ruben        NaN      Torres    NaN   
3           11003   NaN     Christy        NaN         Zhu    NaN   
4           11004   NaN   Elizabeth        NaN     Johnson    NaN   
5           11005   NaN       Julio        NaN        Ruiz    NaN   
6           11006   NaN       Janet          G     Alvarez    NaN   
7           11007   NaN    

In [234]:
print(works_custs_bike_buyer.shape)
works_custs_bike_buyer.head()

(16404, 24)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,BikeBuyer
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,Bachelors,Professional,M,M,1,0,0,2,137947,0
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,Bachelors,Professional,M,S,0,1,3,3,101141,1
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,Bachelors,Professional,M,M,1,1,3,3,91945,0
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,Bachelors,Professional,F,S,0,1,0,0,86688,0
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,Bachelors,Professional,F,S,1,4,5,5,92771,1


In [235]:
labels = np.array(works_custs_bike_buyer['BikeBuyer'])
labels.size

16404

In [236]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education', 'Occupation', 'Gender', 'MaritalStatus']

Features = encode_string(works_custs_bike_buyer['BikeBuyer'])
for col in categorical_columns:
    temp = encode_string(works_custs_bike_buyer[col])
    Features = np.concatenate([Features, temp], axis = 1)

print(Features.shape)
print(Features[:2, :])    

(16404, 16)
[[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1.]]


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [237]:
#Features = np.concatenate([Features, np.array(aw_test_bikebuyer[['HomeOwnerFlag', 'NumberCarsOwned', 
#                            'NumberChildrenAtHome', 'YearlyIncome']])], axis = 1)
Features = np.concatenate([Features, np.array(works_custs_bike_buyer[['NumberChildrenAtHome', 'YearlyIncome']])], axis = 1)
print(Features.shape)
print(Features[:2, :])  

(16404, 18)
[[1.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00
  0.00000e+00 1.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 1.37947e+05]
 [0.00000e+00 1.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00
  0.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00 3.00000e+00 1.01141e+05]]


In [238]:
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 500) #need to have shuffle=False in here for correct answer
X_train = Features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
#X_test = Features[indx[1],:]
#y_test = np.ravel(labels[indx[1]])

In [241]:
scaler = preprocessing.StandardScaler().fit(X_train[:,17:])
X_train[:,17:] = scaler.transform(X_train[:,17:])
#X_test[:,17:] = scaler.transform(X_test[:,17:])
X_train[:2,]

array([[ 1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        , -0.52751147],
       [ 1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        , -0.44654991]])

In [242]:
#logistic_mod = linear_model.LogisticRegression() 
#logistic_mod.fit(X_train, y_train)

In [243]:
#print(logistic_mod.intercept_)
#print(logistic_mod.coef_)

In [244]:
#probabilities = logistic_mod.predict_proba(Features)
#print(probabilities[:15,:])

In [245]:
#def score_model(probs, threshold):
#    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
#scores = score_model(probabilities, 0.5)
#print(np.array(scores[:500]))

In [246]:
# Start the test data prediction
aw_test = pd.read_csv('aw_test.csv')
aw_test.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
print(aw_test.columns)

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome'],
      dtype='object')


In [247]:
labelsT = np.array(aw_test['Occupation'])
lavelsT.size()

NameError: name 'lavelsT' is not defined

In [248]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education', 'Gender', 'MaritalStatus']

FeaturesT = encode_string(aw_test['Occupation'])
for col in categorical_columns:
    tempT = encode_string(aw_test[col])
    FeaturesT = np.concatenate([FeaturesT, tempT], axis = 1)

print(FeaturesT.shape)
print(FeaturesT[:2, :])

(500, 14)
[[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0.]]


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [249]:
FeaturesT = np.concatenate([FeaturesT, np.array(aw_test[['HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'YearlyIncome']])], axis = 1)
#FeaturesT = np.concatenate([Features, np.array(aw_test[['HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'YearlyIncome']])], axis = 1)
print(FeaturesT.shape)
print(FeaturesT[:2, :]) 

(500, 18)
[[0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00
  0.00000e+00 1.00000e+00 0.00000e+00 2.00000e+00 0.00000e+00 8.69310e+04]
 [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 1.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00
  1.00000e+00 0.00000e+00 1.00000e+00 2.00000e+00 2.00000e+00 1.00125e+05]]


In [250]:
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(FeaturesT.shape[0])
indx = ms.train_test_split(indx, test_size = 499, shuffle=False) #need to have shuffle=False in here for correct answer
#X_train = FeaturesT[indx[0],:]
#y_train = np.ravel(labelsT[indx[0]])
X_test = FeaturesT[indx[1],:]
y_test = np.ravel(labelsT[indx[1]])

In [251]:
scaler = preprocessing.StandardScaler().fit(X_train[:,17:])
#X_train[:,16:] = scaler.transform(X_train[:,16:])
X_test[:,17:] = scaler.transform(X_test[:,17:])
#X_train[:2,]

In [252]:
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [253]:
print(logistic_mod.intercept_)
print(logistic_mod.coef_)

[-0.26951155]
[[-6.90673334  6.63722179  0.07264017  0.01513166 -0.12311974 -0.05429136
  -0.17987228 -0.04919292 -0.29294805  0.01524023  0.01081282  0.04657636
  -0.27822726  0.00871571 -0.48253394  0.21302239  0.34348087  0.21826408]]


In [254]:
print(FeaturesT)

[[0.00000e+00 1.00000e+00 0.00000e+00 ... 2.00000e+00 0.00000e+00
  8.69310e+04]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 2.00000e+00 2.00000e+00
  1.00125e+05]
 [0.00000e+00 1.00000e+00 0.00000e+00 ... 2.00000e+00 0.00000e+00
  1.03985e+05]
 ...
 [0.00000e+00 1.00000e+00 0.00000e+00 ... 4.00000e+00 3.00000e+00
  1.38097e+05]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 1.00000e+00 0.00000e+00
  1.01465e+05]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 2.00000e+00 0.00000e+00
  1.91080e+04]]


In [255]:
probabilities = logistic_mod.predict_proba(X_test)
print(probabilities[:15,:])

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [256]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
scores = score_model(probabilities, 0.5)
print(np.array(scores[:500]))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 